# Mecánica de los Fluidos Computacional
## Algoritmo PISO para solución de las Ecs. de Navier Stokes

In [2]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

El nombre PISO  es un acrónimo de _Pressure Implicit Splitting of Operators_ , es un esquema multipaso desarrollado por R. Issa en 1986 para resolver flujos incompresibles o compresibles estacionarios o inestacionarios. 

El método resuelve de manera discreta el sistema de leyes de conservación  dado en notación indicial por

$$\begin{align}
\frac{\partial \rho }{\partial t} + \frac{\partial \rho u_i }{\partial x_i}  &=0 \\
\frac{\partial \rho u_i }{\partial t} + \frac{\partial \rho u_j u_i }{\partial x_j}  &=-\frac{\partial p}{\partial x_i}+\frac{\partial}{\partial x_j} \sigma_{ij} \\
 \frac{\partial \rho e }{\partial t} + \frac{\partial \rho u_j e }{\partial x_j}  &=\frac{\dot{  \partial q_j}}{\partial x_j}-\frac{\partial p u_j}{\partial x_j}+\frac{\partial}{\partial x_j} u_i \sigma_{ij}
\end{align}
$$

El requisito para resolver flujos compresibles es que la Ec. de estado pueda expresarse de la forma

$$ \begin{equation}
\rho=p \phi(p, t)
\end{equation}$$

donde $\phi$ es la llamada _función de compresibilidad_, que en general es función de la presión y la temperatura. Para un gas ideal es igual a 
$$ \phi= \frac{1}{RT}$$

Las ecuaciones se expresan en función de las variables primitivas $p$, $u_i$ y  $\rho$, lo cual hace que el método no esté bien condicionado para resolver problemas de ondas de choque o discontinuidades. 

Como se trata de un problema no lineal, debe ser resuelto iterativamente hasta llegar a convergencia. Sin embargo, el autor demostró que con un esquema de un paso predictor y dos pasos  correctores se obtiene un error de truncamiento aceptablemente pequeño. En cada uno de los pasos mencionados se obtienen valores intermedios de las variables del problema, que se usan para corregir las subsiguientes.   El algoritmo se resume en los siguientes pasos
1. se define la condición inicial ($p^{(n)}$, $u_i^{(n)}$, $\rho_i^{(n)}$  el estado en el paso de tiempo anterior
2. Se determina un campo de velocidades $u_i^{*}$ mediante integración implícita en función de $p^{(n)}$ con la Ec. de cant. de Mov.
3. Se obtiene un nuevo valor intermedio para la presión $p^{*}$ con $u_i^{*}$ con la Ec. de la presión.
4. Se obtiene un nuevo valor intermedio para la velocidad $u_i^{**}$ con $p^{*}$ 
4. Se obtiene un nuevo valor intermedio para la presión $p^{**}$ con $u_i^{**}$
4. Se obtiene un nuevo valor intermedio para la velocidad $u_i^{***}$ con $p^{**}$


## Flujo Incompresible

En el caso de que el flujo sea incompresible la Ec. de la energía se encuentra desacoplada de las Ecs. de continuidad y Cantidad de Movimiento.El sistema de leyes de conservación se reduce a:

$$\begin{align}
 \frac{\partial  u_i }{\partial x_i}  &=0 \\
\rho \frac{\partial  u_i }{\partial t} +\rho \frac{\partial  u_j u_i }{\partial x_j}  &=-\frac{\partial p}{\partial x_i}+\frac{\partial}{\partial x_j} \sigma_{ij} 
\end{align}
$$
Se tienen tres incógnitas en el caso bidimensional (2 componentes de velocidad y la presión) y cuatro en el caso tridimensional (3 componentes de velocidad y la presión). Como en la Ec. de continuidad no se encuentra la presión y la Ec. de cant de movimiento se emplea para determinar las componentes de la velocidad, es necesario determinar el campo de presiones de una ecuación obtenida de combinar la Ec. de continuidad y la de Cant. de Mov. 
### Ecuaciones discretizadas 

La ecuación de la cantidad de movimiento y la de continuidad pueden expresarse de manera discreta de la forma

$$\begin{align}
\frac{(\rho u_i)^{n+1}-(\rho u_i)^{n}}{\Delta t}& = H (u^{n+1})- \Delta_i (p^{n+1})\\
\frac{\rho^{n+1}-\rho^{n}}{\Delta t} & =- \Delta_i ((\rho u_i)^{n+1})
\end{align}
$$

donde $H(u)$ es un operador de discretización espacial de los flujos convectivos y difusivos;  y $\Delta_i$ es un operador de diferenciación numérica centrado. En general el operador $H(u)$ puede expresarse  como:

$$\begin{equation}
H(u) =\sum_m A_m u_{i+m}
\end{equation}
$$
donde m son los índices de los  vecinos a la celda considerada. Tomando como ejemplo un esquema de volúmenes finitos upwind unidimensional para una discretización para volúmenes de control de velocidad, el operador $H(u^{n+1})$ resulta: 

 $$  H(u) =\left[ D_{i+1/2}+D_{i-1/2}+max(F_{i-1/2}, 0)-min(F_{i+1/2}, 0)-(F_{i+1/2}-F_{i-1/2})\right] u_{i}^{n+1}+\left[ -D_{i+1/2}+min(F_{i+1/2}, 0)\right] u_{i+1}^{n+1} \dots \\
  +\left[  -D_{i-1/2}-max(F_{i-1/2}, 0) \right]u_{i-1}^{n+1}$$
  
   Tomamos la numeración respecto a las celdas de velocidad, para mantener la nomenclatura del paper. El operador diferencia centrada aplicado a la presión resulta 
   
   $$\Delta_i (p^{n+1})=\frac{p_{i+1/2}^{n+1}-p_{i-1/2}^{n+1}}{\Delta x}
   $$
   
   ### Ecuación para la presión
   
   El operador  $\Delta_i$ es el equivalente discreto de la divergencia. Tomando la divergencia de la Ec. de cantidad de movimiento
   $$\begin{equation}
   \frac{\Delta_i(\rho u_i)^{n+1}-\Delta_i(\rho u_i)^{n}}{\Delta t} = \Delta_i H (u^{n+1})- \Delta_i^2 (p^{n+1})
   \end{equation}
   $$
   donde $\Delta_i^2$ es el operador laplaciano discreto. Despejando $\Delta_i(\rho u_i)^{n+1}$ y reemplazando en la Ec. de continuidad se obtiene
$$
  \frac{\rho^{n+1}-\rho^{n}}{\Delta t}+ \Delta_i(\rho u_i)^{n}+{\Delta t}\left[ \Delta_i H (u^{n+1})- \Delta_i^2 (p^{n+1})\right]=0
      $$
   Despejando para la presión
   $$\begin{equation}
 \Delta_i^2 (p^{n+1})= \frac{\rho^{n+1}-\rho^{n}}{\Delta t^2}+ \frac{\Delta_i (\rho u_i)^{n}}{\Delta t}+ \Delta_i H (u^{n+1})
   \end{equation}
   $$
   
   Esta ecuación expresa que la presión se ve modificada por efectos convectivos y difusivos (contenidos en la divergencia de H) y por efectos de desbalance másico (en la divergencia del flujo de cantidad de movimiento). 
   
   De esta manera, se obtiene un sistema de tres ecuaciones acopladas entre sí: 
 - La Ec. de la Cant. de Movimiento discreta (3)
 - la Ec. de la Presión (7)
 - La Ec. de la divergencia nula para $u_i$
 
 En lugar de resolver en conjunto  estas tres ecuaciones de manera acoplada, el autor propone resolverlas de manera secuencial, desacoplándolas. 
   

## Implementación del algoritmo

El algoritmo se implementa con una serie de pasos secuenciales, donde el siguiente da una corrección del resultado obtenido en el anterior. El primer paso es un paso predictor, donde se hace una primera estimación $u^*_i$ con las condiciones del paso anterior

### Primer  Paso predictor

Se resuelve la Ec. de cantidad de movimiento implícita 

$$\begin{equation}
\frac{\rho}{\Delta t}(u_i^*-u_i^n)= H(u_i^*)-\Delta_i p^n
\end{equation}
$$

El valor obtenido de $u_i^*$ en general no cumplirá con la condición de divergencia nula, salvo que el campo de presiones $p^n$ sea  solución exacta. Para ello es necesario corregir la solución para obtener una distribución de presiones y velocidades que satisfagan $ \nabla \cdot u=0$

###  Primer Paso corrector

Se debe encontrar un nuevo campo de velocidades $u_i^{**}$ y un campo de presiones $p^*$ que satisfagan

$$ \Delta_i u_i^{**}=0 
$$

Para ello se evalúa la Ec. de cantidad de movimiento de manera explícita, evaluando el operador de los flujos convectivos y difusivos con $u_i^*$

$$\begin{equation}
\frac{\rho}{\Delta t}(u_i^{**}-u_i^n)= H(u_i^*)-\Delta_i p^*
\end{equation}
$$

Nuevamente, tomando la divergencia discreta a dicha ecuación y reemplazando $\Delta_i u_i^{**}=0 $ se obtiene un sistema de ecuaciones  para $p^*$.

   $$\begin{equation}
 \Delta_i^2 (p^{*})=  \frac{ \rho }{\Delta t}\Delta_i u_i^{n}+ \Delta_i H (u^{*})
   \end{equation}
   $$
   
   Una vez obtenida $p^*$ se reemplaza en (2) para obtener $u_i^{**}$
   $$\begin{equation}
u_i^{**}=u_i^n+ \frac{\Delta t}{\rho}\left[ H(u_i^*)-\Delta_i p^* \right]
\end{equation}
$$

### Segundo paso corrector

Para evitar acumulación de errores por imbalance de masa se propone nuevo campo de velocidades $u_i^{***}$ y un campo de presiones $p^{**}$ que también satisfagan la divergencia discreta nula.Se repite el mismo razonamiento que para el caso anterior: planteando una nueva Ec. de cantidad de movimiento evaluada en  $u_i^{**}$

$$\begin{equation}
\frac{\rho}{\Delta t}(u_i^{***}-u_i^n)= H(u_i^{**})-\Delta_i p^{**}
\end{equation}
$$
Se obtiene así una Ec. de Laplace para $p^{**}$

   $$\begin{equation}
 \Delta_i^2 (p^{**})=  \frac{ \rho }{\Delta t}\Delta_i u_i^{n}+ \Delta_i H (u^{**})
   \end{equation}
   $$
  Y se resuelve para el tercer campo de velocidades 
     $$\begin{equation}
u_i^{***}=u_i^n+ \frac{\Delta t}{\rho}\left[ H(u_i^{**})-\Delta_i p^{**}\right]
\end{equation}
$$ 

### Avance en el tiempo

La integración temporal se hace simplemente asignando

$$\begin{align}
u_i^{n+1}& = u_i^{***}\\
p^{n+1}& = p^{**}
\end{align}
$$

En principio podrían hacerse más pasos de corrección para la presión, pero para la mayoría de los problemas de ingeniería con 3 correcciones para la velocidad resulta suficiente para tener valores aceptables de error de truncamiento. 



In [ ]:
"""
solución de ec. de navier stokes incompresibles  1D por el esquema PISO
"""


"""
vectores de parámetros
"""
ρ=numpy.ones(nx+2)#ditribucion de densidad
μ=numpy.ones(nx+2)*1e-4 #viscosidad 
A=numpy.ones(nx+2) #distribución de areas
d= numpy.ones(nx)
"""
    Guess inicial para presión y velocidad
"""
pstar=numpy.ones(nx)# campo de presiones propuesto 
ustar= numpy.ones(nx)# campo de velocidades propuesto
pprime=numpy.ones(nx)# corrección al campo de presiones propuesto 
uprime=numpy.ones(nx)# corrección al campo de velocidades propuesto 

"""
condiciones de contorno
"""

p0=1; pL=1;

"""
asignación de memoria para variables
"""

M_vel=numpy.zeros((nx, nx)) # matriz de coefs de influencia para la velocidad u
RHS_vel=numpy.zeros(nx)

M_p=numpy.zeros((nx, nx)) # matriz de coefs de influencia para la presion
RHS_p=numpy.zeros(nx)
"""
sistema de ecuaciones para la velocidad empleando el esquema upwind
"""
# convención de índices 
#se usa I para la velocidad, donde I=i+1/2

#valores en el contorno
D_0=2*μ[0]/dx;    D_L=2*μ[nx+1]/dx; 
F_0=ρ[0]*u[0];    F_L=ρ[nx+1]*u[nx+1];

# BC izquierda
u_3o2=(u[0]+u[1])/2  #u_3/2

D_1=μ[1]/dx
F_1o2=0;  F_3o2=(ρ[1]+ρ[2])/2*u_3o2;

M_vel[0][0] =-min(F_0, 0)+max(F_3o2,0)+D_3o2+D_0
M_vel[0][1]   =-D_3o2- min(F_3o2,0)
d[0]=A[0]/M_vel[0, 0]/dx

RHS[0] = (pL-pstar[0])*A[i]/dx
#ultima fila de la matriz
u_nm3o2=(u[nx-2]+u[nx-1])/2 #u_n-1/2

D_nm1=μ[nx-1]/dx
F_nm3o2=(ρ[nx-1]+ρ[nx])/2*u_nm3o2;  

M_vel[nx-1][nx-1] =max(F_L, 0)-min(F_nm3o2, 0)+D_nm1o2+D_L
M_vel[nx-1][nx-2]   =-D_nm1o2-F_nm1o2/2
d[nx-1]=A[nx-1]/M_vel[nx-1][nx-1]/dx
RHS[nx-1] = (pstar[nx-1]-pL)*A[i]/dx

for i in range(1, nx-1):
    #flujos convectivos en los centros de las celdas de velocidad
    I=i
    F_ip3o2=(ρ[i+2]+ρ[i+1])/2 *ustar[I+1];
    F_ip1o2=(ρ[i]+ρ[i+1])/2 *ustar[I];
    F_im1o2=(ρ[i]+ρ[i-1])/2 *ustar[I-1];  
    F_im3o2=(ρ[i-1]+ρ[i-2])/2 *ustar[I-2]; 
    #flujos en las interfaces en celdas de velocidad
    F_i  = (F_ip1o2+F_im1o2)/2
    F_ip1 =(F_ip3o2+F_ip1o2)/2
    F_im1 =(F_im1o2+F_im3o2)/2
    #flujos difusivos
    
    D_im1=μ[i-1]/dx;     D_i=μ[i]/dx; D_ip1=μ[i+1]/dx;
    
    M_vel[i][i-1] = (-D_im1-max(F_im1, 0));
    M_vel[i][i+1] = (-D_ip1+min(F_ip1, 0));
    M_vel[i][i]   =  D_im1+D_i+max(F_i, 0)-min(F_ip1, 0)-(F_ip1-F_i);
    d[i]=A[i]/M_vel[i][i]/dx
    RHS_vel[i]=(pstar[i]-pstar[i+1])*A[i]/dx

u_star=numpy.linalg.solve(M_vel, RHS_vel) #nuevo valor de u* despuès de desolver la ec. de cant de mov


"""
cálculo de la corrección de la presión
"""
ρ_3o2=(ρ[1]+ρ[2])/2 
ρ_1o2=(ρ[0]+ρ[1])/2

M_p[0][1] = -ρ_3o2*A[1]*d[1];
M_p[0][0]   =  ρ_3o2*A[1]*d[1]+ρ_1o2*A[0]*d[0];
RHS_p[i]=ρ_1o2*A[0]*d[0]*p0

ρ_nm3o2=(ρ[nx-1]+ρ[nx-2])/2 
ρ_nm1o2=(ρ[nx-1]+ρ[nx])/2

M_p[nx-1][nx-2] = -ρ_nm3o2*A[nx-2]*d[nx-2];
M_p[nx-1][nx-1]   =  ρ_nm3o2*A[nx-2]*d[nx-2]+ρ_nm1o2*A[nx-1]*d[nx-1];
RHS_p[nx-1]=ρ_nm1o2*A[nx]*d[nx]*pL


for i in range(1, nx-1):
    #flujos convectivos en las interfaces de las celdas de presión
    I=i

    ρ_ip1o2=(ρ[i]+ρ[i+1])/2 
    ρ_im1o2=(ρ[i]+ρ[i-1])/2   

    #flujos difusivos
    
    M_p[i][i-1] = -ρ_im1o2*A[i-1]*d[i-1];
    M_p[i][i+1] = -ρ_ip1o2*A[i]*d[i];
    M_p[i][i]   =  ρ_ip1o2*A[i]*d[i]+ρ_im1o2*A[i-1]*d[i-1];
    
    RHS_p[i]= -ρ_im1o2*A[i]*u_star[i]+ρ_im1o2*A[i-1]*u_star[i-1]
p_prime=numpy.linalg.solve(M_p, RHS_p)
                    
pyplot.plot(x_vel[0:nx], u_star,  lw=0.5 , marker='+')
pyplot.plot(x_pres[0:nx], p_prime,  lw=0.5 , marker='+')
pyplot.xlabel("x")
pyplot.ylabel("φ")
#pyplot.gca().legend(('celdas presion','celdas u'))
pyplot.grid(b=None, which='major', axis='both')
